In [532]:
import pandas as pd
import numpy as np
import os

#load odds and results
data = pd.read_csv('odds&results2024.csv')
abvr_match = pd.read_csv('team_abbrv_match.csv')
data = data.merge(abvr_match, left_on='home_team', right_on='team_name')
data = data.merge(abvr_match, left_on='away_team', right_on='team_name')
odds_and_results = data.rename(columns={"team_abbrv_x": "home_team_abbrv", "team_abbrv_y": "away_team_abbrv"})
#odds_and_results.head(3)

#parse through raw output and add to full_list dataframe
player_list = [37,'Billiard91','Bob Z','EllieisAwesome1','JJB','JJapcon','Jack Ballard','K-Dog','Kathy Zwartz','Kkrugz','Linds Domino','Lizzy_Z','Matt McKenna','MattWojtanek','Michael Zwartz','Midway Monsters','Murr-auders','PBW','Shan-diddy','Shaner','Shumarie','TAZ','Team Boylan','TeamGalarza1020','UrBoiJZ','Zapp','abfarias','taz7611']
full_picks = pd.DataFrame() 

#parse all results for clean data file
directory = os.fsencode('weekly_data\picks2024')
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    nfl_week = (filename.split(".")[0]).split("week")[1]
        
    df = pd.read_excel(f"weekly_data\picks2024\{filename}")
    df.columns.values[0] = "player" #rename column for merge

    #fill in black player names in points rows
    df.replace(" ", np.nan, inplace=True)
    df['player'] = df['player'].ffill()

    #add 'player' for home team and drop other game rows
    df['player'].fillna(value='home_team', inplace=True)
    df = df.drop(index=[0,1], columns=df.columns[-1])

    #reverse rows and columns
    df1 = df.transpose()

    # Make the first row the new column names
    df1.columns = df1.iloc[0]  # Set the new column names
    df1 = df1[1:]  # Drop the first row
    df1 = df1.reset_index(drop=True)
    df2 = df1.copy()
    df2['nfl_week'] = int(nfl_week)
    
    for player in player_list:
        if player in df2.columns:
            player_df = df2[['home_team', 'nfl_week', player]].copy()
            player_df['player'] = player
            player_df.columns.values[2] = "pick"
            player_df.columns.values[3] = "points"
            
            #append each player to the full list        
            full_picks = pd.concat([full_picks, player_df])
        
data_merged = full_picks.merge(odds_and_results, left_on=['home_team', 'nfl_week'], right_on=['home_team_abbrv', 'nfl_week'], how='left')


In [533]:
i = 1

while i <= max(odds_and_results['nfl_week']): 
    #calculate number of upsets 
    upset_table = odds_and_results[odds_and_results['nfl_week']==i].copy()

    # Count upsets
    upset_condition = ((upset_table['home_spread'] > 0) & (upset_table['home_score'] > upset_table['away_score'])) | \
                      ((upset_table['home_spread'] < 0) & (upset_table['home_score'] < upset_table['away_score']))

    upset_count = upset_condition.sum()

    print("Number of upsets in week", i, "-", upset_count)
    i = i + 1


Number of upsets in week 1 - 3
Number of upsets in week 2 - 8
Number of upsets in week 3 - 8
Number of upsets in week 4 - 7
Number of upsets in week 5 - 3
Number of upsets in week 6 - 0


In [569]:
#create a measure for pick success (points given * point differential)
total_score = data_merged[['game','player','pick','points','nfl_week','home_team_abbrv','home_spread','away_team_abbrv','home_score','away_score']].copy()

# Create the 'winner' column
total_score['winner'] = np.where(total_score['home_score'] > total_score['away_score'], 
                        total_score['home_team_abbrv'], 
                        total_score['away_team_abbrv'])

total_score['point_diff'] = abs(total_score['home_score'] - total_score['away_score'])

# Create the 'score' column
total_score['score'] = np.where(total_score['pick'] == total_score['winner'], total_score['points'], 0)
total_score['pick_rating'] = np.where(total_score['pick'] == total_score['winner'], 
                                     total_score['points']*total_score['point_diff'],
                                     total_score['points']*-total_score['point_diff'])

results = total_score.groupby(['player', 'nfl_week'])[['score', 'pick_rating']].sum().reset_index()
#results[results['nfl_week'] == 6].sort_values('score', ascending=False)
results.head()

,player,nfl_week,score,pick_rating
0,37,1,115,1193.0
1,37,2,60,-55.0
2,37,3,57,153.0
3,37,4,79,217.0
4,37,5,70,552.0


In [ ]:
from scipy.stats import norm

pick_strength = results.copy()
pick_strength['z_score'] = 0

week = max(results['nfl_week'])
while week > 0:
    week_avg = pick_strength['pick_rating'][pick_strength['nfl_week']==week].mean()
    week_std = pick_strength['pick_rating'][pick_strength['nfl_week']==week].std()
    
    pick_strength['z_score'].loc[pick_strength['nfl_week'] == week] = (pick_strength['pick_rating'] - week_avg)/week_std
    week = week - 1
    
pick_strength['pick_strength_rtg'] = norm.cdf(pick_strength['z_score']) * 100



In [628]:
pick_strength[['player', 'score', 'pick_strength_rtg']][pick_strength['nfl_week'] == 6].sort_values('score', ascending=False).head(10)

,player,score,pick_strength_rtg
107,PBW,133,79.383503
136,Team Boylan,133,82.481913
154,Zapp,130,87.506273
160,abfarias,129,71.834593
83,MattWojtanek,126,62.295530
77,Matt McKenna,125,79.771602
59,Kkrugz,124,85.786872
101,Murr-auders,123,76.194037
17,Bob Z,121,82.422994
142,TeamGalarza1020,119,72.675698
